# Session #1 
## Dictionary-based SA using SentiStrength dictionary

첫 번째 세션에서는 SentiStrength에 탑재된 사전을 활용하여 간단한 사전 기반 (dictionary-based) 영어 텍스트 감성분석 프로그램을 작성해 봅니다.  
실습 수업은 프로그램의 주요 흐름을 설명하면서, TODO 처리된 핵심적인 부분의 코드를 직접 작성해보는 순서로 진행될 것입니다.  

사전 데이터와 테스트에 사용할 텍스트 파일이 있는 폴더를 지정합니다.  
여기서는 사전 데이터가 'SentiStrength_Data' 폴더에, 텍스트 파일이 '6humanCodedDataSets' 폴더에 있는 것을 전제로 하였습니다.  
만일 사전 데이터와 텍스트 파일이 다른 폴더에 있을 경우 이를 변경하셔야 합니다.  
(코드 .ipynb 파일과 같은 폴더에 있을 시에는 './'로 설정하여 주십시오)

### **구글 드라이브를 이용하여 데이터 import 및 압축풀기**
a) 내 구글 드라이브 mount 하기


In [1]:
""" 
내 구글 드라이브와 colab 연결 ==> 
아래 셀 실행 후 출력되는 링크 클릭 ==> 
authorization code 복사 후 아래 출력된 박스에 붙여넣고 enter 키 입력
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
내 구글 드라이브와 잘 연동 되었는지 확인해보기
실습 파일들을 내 구글 드라이브 (My Drive) 내에 다른 폴더를 만들었다면 ==> 
os.listdir('/content/drive/My Drive/내가_만든_폴더_이름')
"""
import os
os.listdir('/content/drive/My Drive/Colab Notebooks/20.10.30')

['6humanCodedDataSets.zip',
 'SentiStrength_Data.zip',
 'data.zip',
 'session1_dictionary_SA.ipynb',
 'session2_NaiveBayes_SA.ipynb',
 '2020_2학기_assignment_session2_NaiveBayes_SA.ipynb',
 '2020_2학기_assignment_session1_dictionary_SA.ipynb']

In [3]:
"""
구글 드라이브에 올려놓은 실습 데이터 zip 압축 풀기 ==> 
zipfile 의 extractall 함수 이용 
압축을 풀 경로 (directory_to_extract_to)는 반드시 '/tmp/' 아래 지정할 것 ex) '/tmp/SentiStrength_Data/'
"""
import zipfile
path_to_zip_file = '/content/drive/My Drive/Colab Notebooks/20.10.30/SentiStrength_Data.zip'
directory_to_extract_to = '/tmp/SentiStrength_Data'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)

path_to_zip_file = '/content/drive/My Drive/Colab Notebooks/20.10.30/6humanCodedDataSets.zip'
directory_to_extract_to = '/tmp/6humanCodedDataSets'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)

In [4]:
"""
압축이 잘 해제 되었는지 확인하기
"""
print(os.listdir('/tmp/SentiStrength_Data'))
print(os.listdir('/tmp/6humanCodedDataSets'))

['SlangLookupTable.txt', 'IdiomLookupTable.txt', 'BoosterWordList.txt', 'NegatingWordList.txt', 'EmotionLookupTable.txt', 'EnglishWordList.txt', 'EmoticonLookupTable.txt', 'QuestionWords.txt']
['digg1084.txt', 'YouTube3407.txt', 'rw1046.txt', '1041MySpace.txt', 'bbc1000.txt', 'twitter4242.txt']


In [5]:
"""
압축을 푼 경로로 데이터 경로 지정
"""
dict_dir = '/tmp/SentiStrength_Data/'
data_dir = '/tmp/6humanCodedDataSets/'

### Step 1. Tokenizing
문장을 먼저 단어 단위로 분할하는 작업을 수행합니다. 영어 텍스트인 만큼 기본적으로 띄어쓰기 단위로 분할을 하되, 문장 부호는 삭제하고, 대문자는 소문자로 치환하도록 합니다.  
여기서는 정규 표현식과 re.sub() 함수를 활용하여 문장 부호 (? ! " . , ;)를 제거하는 부분을 직접 작성해봅시다.  
작성 후 함수를 직접 실행하여 의도대로 구현되었는지 확인합니다.

In [6]:
'''
Parse a given sentence to a list
Input: sentence string
Output: List of words
'''
def parse_input(str):
    import re   #import regular expressions module
    # TODO : remove punctuations using regular expression and re.sub() function
    # for example, using str = re.sub(r'[abc]','',str) to remove a character a, b or c from string str.
    
    str = re.sub(r'[!?";,.]','',str)
    str = str.lower() #make all characters lowercase
    words = str.split(' ') #split words in string on space and save them in a list.
    
    return words

In [7]:
parse_input('Why the deafening silence?  It shows the duplicity of "the allies".')

['why',
 'the',
 'deafening',
 'silence',
 '',
 'it',
 'shows',
 'the',
 'duplicity',
 'of',
 'the',
 'allies']

### Step 2. Parsing dictionaries
여기에서는 감성분석에 사용할 각 사전들을 읽어옵니다.
사전은 크게 2가지 형태가 사용되는데, 하나는 각 단어마다 고유한 weight가 부여된 사전이며, 나머지 하는 부정어 (negating word)가 저장된 사전인데 부정어의 경우 별도의 weight를 가지지 않으므로 부정어 사전의 모든 weight는 0이 됩니다.
그래서 각각에 대해 별도의 사전 생성 함수를 작성할 것입니다. Python의 dictionary 자료형을 이용합니다.

1) weight가 저장된 dictionary를 불러오는 함수입니다.  
해당 사전들은 매 줄이 "단어 (탭) weight (탭) 주석" 형식으로 저장되어 있습니다.  
이를 읽어서 저장하는 코드를 작성합니다.

In [8]:
'''
Parse boosting weights
Input: file name
Output: dictionary of weights
'''
def parse_weight(fname):
    w = {}   #dictionary
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            # TODO : add a dictionary entry from each line
            # You first split the line into a word, a weight and a comment, then add the entry to the dictionary
            # Hint : you may use the split() function

            rule = line.split('\t')  #split words in string on tab and save them in a list
            w[rule[0]] = int(rule[1]) #the first component of the list becomes 'key', the second one becomes 'value' of the dictionary.
            
    return w

2) negating word에 대한 dictionary를 불러오는 함수입니다.  
negating word 사전은 매 줄 마다 단어만 저장되어 있습니다.  
따라서 weight는 모두 0으로 저장합니다. 이를 감안하여 함수를 작성합니다.

In [9]:
'''
Parse negations
Input: file name
Output: dictionary of weights (all 0)
'''
def parse_negate(fname):
    w = {}   #dictionary
    with open(fname, 'r') as f :
        for line in f: #for each line in the file
            # TODO : add a dictionary entry from each line
            # Unlike other dictionaries, the negating word dictionary only has a word in each line
            # An weight for each line should be assigned as '0'.

            w[line[:-1]] = 0  #each word becomes 'key' and '0' becomes 'value' of the dictionary.(exclude the last component '\n')
            
    return w

실제 사전 데이터를 통해 구현된 함수를 확인합니다.  
실행 결과는 2, 0 으로 나와야 합니다. (각각 extremely와 never에 대한 weight)  
필요한 경우 다른 단어에 대해 사전을 확인해 보실 수도 있습니다. (예 : dict_boost['very'])

In [10]:
dict_boost = parse_weight(dict_dir + 'BoosterWordList.txt')
dict_negate = parse_negate(dict_dir + 'NegatingWordList.txt')
print('%d, %d'%(dict_boost['extremely'],dict_negate['never']))

2, 0


### Step 3. Weighting
구축한 dictionary를 이용해 주어진 문장의 positive/negative strength (weight)를 찾아냅니다.  
즉 Parsing한 단어들의 weight를 주어진 모델의 dictionary에서 찾는 과정입니다.  
사전에는 'ador*' 와 같이 어두에 대한 정의가 존재하므로 전체 단어가 사전에 없어도 어두를 통해 weight를 부여하는 경우도 고려해야 합니다.  
이를 위해 단어가 사전에 존재하지 않는 경우 끝에서부터 1 글자씩 잘라가면서 어두를 찾는 루틴도 구현합니다.  
그렇게 해도 사전에 존재하지 않는 단어는 0으로 처리합니다.

In [11]:
'''
Give weights on words
Input: list of words, dictionary of weights
Output: list of (word, weight) pairs
'''
def weight_default(list_words, dic_weights):
    l = []  #list
    for word in list_words:  #for each word in the given list,
        if word in dic_weights:          
            l.append((word, dic_weights[word]))  #add word and its weight if the word is in the dictionary
        else:       #if the word is not in the dictionary, check if the words
            substr = word  #first, we define substr as the word
            while substr != '':  #while substr is not NULL,
                pat = substr + '*'   #check if substr* is in the dictionary
                if pat in dic_weights:   
                    l.append((word, dic_weights[pat]))
                    break   #break means we stop the loop and move on to the next word.
                else:
                    # TODO : trim the last character of substr before the loop iterates

                    substr = substr[:-1]   #if substr* is not in the dictionary, we exclude the last character of substr and repeat the above procedure.
                    
            else: # no matching word found in the dictionary
                l.append((word, 0))  #if a matching word is not exist on the dictionary, set its weight as '0'
    return l

### Step 4. Evaluation
본 실습에서는 문장이 부정적/긍정적 감성인지 판단할 때 문장의 각 단어가 가지는 positive strength의 최대값과 negative strength의 최소값을 비교하여, 절대값의 대소에 따라서 positive / neutral / negative를 평가합니다.  
가령 positive의 최대값이 4이고, negative의 최소값이 -3이면 그 문장을 positive sentiment를 가졌다고 평가합니다.  
반면 positive의 최대값이 3이고 negative의 최소값이 -4이면 negative, 두 절대값이 서로 같으면 neutral로 평가해야 합니다.  
아래 extract_max() 함수를 작성하여 (word, weight)의 list가 들어왔을 때 (positive의 최대값, negative의 최소값)을 반환하도록 합니다.

In [12]:
'''
Extract maximum weights for given (word, weight) pairs
Input: list of (word, weight)
Output: (positive, negative)
'''
def extract_max(list_pairs):
    pos_max = 1  #set default of the maximum positive strength as '1'
    neg_max = -1  #set default of the maximum negative strength as '-1'

    for p in list_pairs:
        ## TODO : Iterate over the list of (word, weight) pairs, and extract the max of positive weights and min of negative weights.

        w = p[1]
        if w > 0 and w > pos_max:  #if the weight of the given word is positive and larger than previous maximum,
            pos_max = w            #save it as new maximum positive strength.
        elif w < 0 and w < neg_max: #if the weight of the given word is negative and smaller than previous maximum,
            neg_max = w            #save it as new maximum negative strength.
        

    return (pos_max, neg_max)

### Step 5. Summing up
앞서 구현한 함수들을 종합해 문장을 입력받아 sentiment strength를 출력 및 감성 평가를 수행하는 루틴을 구현합니다.

In [13]:
sentence = 'Why the deafening silence?  It shows the duplicity of "the allies".'
words = parse_input(sentence)

# TODO : parse the emotion dictionary by parsing 'EmotionLookupTable.txt' on the dictionary folder.
dict_emotion = parse_weight(dict_dir + 'EmotionLookupTable.txt')

# TODO : perform a default weighting using the emotion dictionary parsed above.
words_and_weights = weight_default(words, dict_emotion)

for pair in words_and_weights :
    print('(%s : %d)'%(pair[0],pair[1]))

pos_max, neg_min = extract_max(words_and_weights)
print('Max. of positive strength : %d, Min. of negative strength : %d'%(pos_max,neg_min))

# TODO : make a final decision by comparing the absolute value of pos_max and neg_max
if abs(pos_max) > abs(neg_min) :
    print('Positive sentence.')
elif abs(pos_max) < abs(neg_min) :
    print('Negative sentence.')
else :
    print('Neutral sentence.')

(why : 0)
(the : 0)
(deafening : -2)
(silence : 0)
( : 0)
(it : 0)
(shows : 0)
(the : 0)
(duplicity : 0)
(of : 0)
(the : 0)
(allies : 0)
Max. of positive strength : 1, Min. of negative strength : -2
Negative sentence.


### Step 6. Adding rules
사전을 추가하여 감성분석이 좀 더 정밀하게 이루어질 수 있게 weight를 부여하는 룰을 추가하고자 합니다.

#### 6-1. Boosting words
뒤이은 단어의 의미를 강조하는 단어들로, 뒷 단어의 weighting을 강화하거나 약화시키는 룰이 미리 정의되어 있습니다.  
weight_boost() 함수를 통해 이를 반영하고자 합니다. 앞서 weight_default()를 통해 나온 (word, weight)의 list를 입력받아 boosting 된 (word, weight)의 list를 반환해야 합니다.

In [14]:
'''
Give boosting to another word
Input: list of (word, weight)
Output: list of (word, weight)
'''
def weight_boost(list_pairs, dic_boost):
    l = []
    boost = 0    #set default of the boost as '0'
    for p in list_pairs:  #for each (word, weight) in the given list,
        w = p[1]   # save its weight
        if boost != 0:  #if the previous word is the boosting word,
            # TODO : strengthen the weight of current word and add to the words list.
            # Note that positive weights should be increased (+boost)
            # and negative one should be decreased (-boost).
            
            if w > 0:   #strengthen each weight
                w += boost
            else:
                w -= boost
            boost = 0  #reset the boost as its default value
            l.append((p[0], w))  #save the word and its changed weight into the output list.
            
        else:
            l.append(p)  #if the previous word is not the boosting word, save the original (word, weight) into the output list.
        if p[0] in dic_boost:  #check if the word is boosting word
            boost = dic_boost[p[0]] #save the boosting weight
    return l

앞서 boost dictionary를 읽어들였으므로, 이를 사용해 구현된 weight_boost가 잘 동작하는 지 확인합니다.  

In [15]:
sentence = 'i just want to know where you got that totally awesome wallpaper.'
words = parse_input(sentence)
words_and_weights = weight_default(words, dict_emotion)
# You can see that how weight_boost() method works by commenting out following line
words_and_weights = weight_boost(words_and_weights, dict_boost)
for pair in words_and_weights :
    print('(%s : %d)'%(pair[0],pair[1]))

(i : 0)
(just : 0)
(want : 1)
(to : 0)
(know : 0)
(where : 0)
(you : 0)
(got : 0)
(that : 0)
(totally : 0)
(awesome : 4)
(wallpaper : 0)


#### 6-2. Negating words
뒤이은 단어를 부정하는 의미를 추가하는 단어들로, 뒤이인 sentiment word의 strength를 반전시킵니다.  
여기서는 뒤이은 단어가 positive strength를 가질 경우 -0.5를 곱하고, negative인 경우 0으로 만들도록 함수를 작성하시겠습니다.  

In [16]:
'''
Negate the weight for a word
Input: list of (word, weight)
Output: list of (word, weight)
'''
def weight_negate(list_pairs, dic_negate):
    l = []
    negate = 0   #set default of the negation as '0'
    for p in list_pairs:  #for each (word, weight) in the given list,
        w = p[1]  #save its weight
        if negate != 0: #if the previous word is the negating word, apply negating rules
            # TODO : multiply -0.5 for positivie weight, set to 0 for negative weight
            # Note that the weight with fractional component (e.g. +1.5) is not allowed, so you should round it.

            if w > 0:    #for positive weight, we multiply -0.5
                if w % 2 == 1:  #if the weight is odd number
                    w = (w + 1) * -0.5
                else:  #if the weight is even number
                    w *= -0.5
            else:
                w = 0    #for negative weight, we set the weight as 0

            negate = 0   #reset the negation as its default value
            l.append((p[0], int(w)))  #save the word and its changed weight
        else:
            l.append(p)  #if the previous word is not the negating word, save the original (word, weight) into the output list.
        if p[0] in dic_negate: #check if the word is negating word
            negate = 1  #if it is, set the negation as '1' just to indicate this case(you can choose other integers except '0'.)
    return l

마찬가지로 weight_negate가 잘 작동하는 지 확인해봅니다.

In [17]:
sentence = 'I don\'t like to say about such a boring thing.'
words = parse_input(sentence)
words_and_weights = weight_default(words, dict_emotion)
# You can see that how weight_negate() method works by commenting out following line
words_and_weights = weight_negate(words_and_weights, dict_negate)
for pair in words_and_weights :
    print('(%s : %d)'%(pair[0],pair[1]))

(i : 0)
(don't : 0)
(like : -1)
(to : 0)
(say : 0)
(about : 0)
(such : 0)
(a : 0)
(boring : -2)
(thing : 0)


### Step 7. Wrapping up
지금까지 구현한 weight 부여/변형 규칙으로, 텍스트 파일이 주어졌을 때 감성분석을 수행하는 루틴을 작성해 봅니다.  
프로그램 작성을 용이하게 하기 위해 감성분석 값을 반환하는 함수를 먼저 정의합니다.

In [18]:
'''
Our target
Input: Some sentence
Output: Positive, Negative strength
'''
def senti(str):    #this function is to apply above functions to the given string(str).
    
    # Parsing Rules
    w_emotion = parse_weight(dict_dir + 'EmotionLookupTable.txt')  
    w_boost = parse_weight(dict_dir + 'BoosterWordList.txt')
    w_negate = parse_negate(dict_dir + 'NegatingWordList.txt')
    
    # Parse Input
    words = parse_input(str)
    
    # Weighting
    default = weight_default(words, w_emotion)
    boosted = weight_boost(default, w_boost)
    negated = weight_negate(boosted, w_negate)
    
    return extract_max(negated)

다음으로는 지정된 파일에서 문장들을 읽어들여서 감성 분석을 수행하고, 파일에 이미 기록된 결과값 (사람이 직접 평가한 점수)와 비교하는 프로그램을 작성합니다.  
텍스트 파일을 읽을 때 주의사항은 첫 번째 라인은 건너뛰고 감성 분석을 수행하셔야 합니다.

In [19]:
import codecs

textFileName = data_dir + 'bbc1000.txt'
scores = []

with codecs.open(textFileName, 'r', encoding="utf-8") as f :
    # TODO : for each line on the text file, run the senti() function 
    # and print the human evaluated and calculated scores.
    # Note that the 1st line of each text file contains column names for each field, so this should be ignored.
    init = 0
    fl =f.readlines()
    for line in fl[1:]:
      
        s = line.split('\t')
        str = s[2]
        result = senti(str)
        scores.append([int(s[0]), -int(s[1]), result[0], result[1]])    

텍스트 파일 전체에 대한 결과를 프린트할 경우 너무 많아질 수 있기 때문에 숫자를 입력받아 일정 문장만큼만 출력하도록 합니다.

In [20]:
num_to_print = 10

print('(Human evaluated pos / neg), (our pos / neg)')
for i in range(num_to_print) :
    print('(%d/%d),(%d/%d)'%(scores[i][0],scores[i][1],scores[i][2],scores[i][3]))

(Human evaluated pos / neg), (our pos / neg)
(1/-3),(3/-4)
(1/-3),(2/-1)
(1/-2),(1/-3)
(2/-3),(3/-3)
(1/-4),(1/-3)
(1/-1),(1/-1)
(1/-1),(1/-1)
(1/-2),(1/-2)
(2/-2),(2/-2)
(1/-2),(2/-4)
